In [1]:
import numpy as np
import time
import torch

from dask.distributed import Client

In [2]:
import torch.nn as nn
import random
import dask

In [3]:
client = Client()

2024-08-19 14:27:46,153 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:38235' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'internal_fake-49323aca11610d48cb85edd563d2c8e4', 'segment_fake-e25dd518e9addf53c7b5d0bb3484e341', 'internal_fake-5615e0387fc89d9ffe0f6e306e58e439', 'internal_fake-4a906c03eb99b8cf7b3a6156d61021df', 'internal_fake-90e21bea548772400abd9e6a653cebc3'} (stimulus_id='handle-worker-cleanup-1724070466.1528811')
2024-08-19 14:27:46,156 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:44639' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'internal_fake-bb93936b9ea8db11eec3185388d76201', 'internal_fake-09bbe17db4a694c821a13710ed7274a2'} (stimulus_id='handle-worker-cleanup-1724070466.1564114')
2024-08-19 14:27:46,161 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:44551' caused the cluster to lose already computed task(s), whic

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 94.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43763,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 94.21 GiB
Comm: tcp://127.0.0.1:44639,Total threads: 2
Dashboard: http://127.0.0.1:36581/status,Memory: 31.40 GiB
Nanny: tcp://127.0.0.1:36247,


In [5]:
def segment_fake(image):
    img = torch.from_numpy(image).to('cuda:0')
    img = img + 100.0
    time.sleep(0.5)
    return img.cpu().numpy()

In [6]:
def dots_fake(image):
    result = image - 100.0
    time.sleep(0.7)
    return result

In [7]:
def internal_fake(segmented_image, dots_data):
    time.sleep(random.random())
    result = segmented_image + dots_data
    return result

In [8]:

def process_images(image1, image2):
    seg_result = segment_fake(image1)
    dots_result = dots_fake(image2)
    internal_result = internal_fake(seg_result, dots_result)
    return internal_result

In [9]:
results = []
for i in range(10):
    image1 = np.random.random((1000, 1000))
    image2 = np.random.random((1000, 1000))
    seg_result = client.submit(segment_fake, image1)
    dots_result = client.submit(dots_fake, image2)
    internal_result = client.submit(internal_fake, seg_result, dots_result)
    results.append(internal_result)


In [13]:
results

[<Future: cancelled, type: numpy.ndarray, key: internal_fake-8054cb6d34f2b103be38e40088b72dea>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-1ab37c575f4f03394dc539d068f4720f>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-19a2d807ff27627c4f49dc9fe75f50e8>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-6be14be71828b6dda51851d29e94f6ca>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-b922faef865e5b74359a26cbbce9f780>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-50da8a7fc113e06b09af594070963dd8>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-04621d8cb96983a2a43bb4c88ee5ab9c>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-28f975fa44fed0e621d110363f290438>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-2877b67f97094bd7225e6f44c1a17b7e>,
 <Future: cancelled, type: numpy.ndarray, key: internal_fake-1f37f6c404367ef97e7751311ec43721>]

In [14]:
for result in results:
    result.release()

In [ ]:
k